In [92]:
import os
import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

In [117]:
def calculate_energy(subband):
    if len(subband.shape) == 1:
        R, C = 1, len(subband)
    else:
        R, C = subband.shape
    return np.sum(np.abs(subband)) / (R * C)

def dwt_feature_extraction(image, levels):
    imagecv = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    coeffs = pywt.wavedec2(imagecv, 'haar', level=levels)
    energy_vector = []

    for level in range(levels + 1):
        for subband in coeffs[level]:
            energy = calculate_energy(subband)
            energy_vector.append(energy)

    return energy_vector

In [118]:
def svd_feature_extraction(image):
    imagecv = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    U, S, VT = np.linalg.svd(imagecv)
    eigen_vector = VT.T[:, 0]

    return eigen_vector

In [119]:
def fuse_vectors(energy_vector, eigen_vector):
    return np.concatenate((energy_vector, eigen_vector))

In [120]:
def Learning_phase(fingerprint_samples):
    database = []

    for fingerprint_sample in fingerprint_samples:
        energy_vector = dwt_feature_extraction(fingerprint_sample[0], 6)
        eigen_vector = svd_feature_extraction(fingerprint_sample[0])

        feature_vector = fuse_vectors(energy_vector, eigen_vector)

        database.append({'feature_vector': feature_vector, 'class': fingerprint_sample[1]})
    return database


In [121]:
def classify_unknown_fingerprint(unknown_fingerprint, database, k_neighbors=3):
    dwt_feature_extraction(fingerprint_sample, 6)
    eigen_vector = svd_feature_extraction(unknown_fingerprint)

    feature_vector = fuse_vectors(energy_vector, eigen_vector)

    X_train = np.array([entry['feature_vector'] for entry in database])
    y_train = np.array([entry['class'] for entry in database])

    knn_classifier = KNeighborsClassifier(n_neighbors=k_neighbors)
    knn_classifier.fit(X_train, y_train)

    predicted_class = knn_classifier.predict([feature_vector])

    return predicted_class[0]

In [122]:
def get_fingerprint_class(img_path):
    filename, _ = os.path.splitext(os.path.basename(img_path))

    subject_id, age, etc = filename.split('_')

    if int(age) <= 12 :
      ageClass = 0
    elif int(age) <= 19:
      ageClass = 1
    elif int(age) <= 25:
      ageClass = 2
    elif int(age) <= 35:
      ageClass = 3
    elif int(age) > 35:
      ageClass = 4
    
    return ageClass

In [115]:
def loading_data(path):
    data = []
    for img in os.listdir(path):
            ageclass = get_fingerprint_class(os.path.join(path, img))            
            data.append([os.path.join(path, img), ageclass])
            if len(data) % 5== 0:
                print(data)
    return data

In [116]:
#load images where age is know
pathToImage = "./Age"

fingerprint_samples = loading_data(pathToImage)
database = Learning_phase(fingerprint_samples)

#load unknown image to classify
#unknown_fingerprint = "unknow_fingerprint.png"
#predicted_class = classify_unknown_fingerprint(unknown_fingerprint, database)

#print(f"The unknown fingerprint is classified as: {predicted_class}")

[['./Age\\1_6_CR.BMP', 0], ['./Age\\2_6_CR.BMP.BMP', 0], ['./Age\\3_15_CR.BMP', 1], ['./Age\\4_66_CR.BMP.BMP', 4], ['./Age\\5_15_CR.BMP.BMP', 1]]


C:\Users\pc\AppData\Local\Temp\ipykernel_13972\2761517714.py:4: RuntimeWarning: divide by zero encountered in divide
  return np.sum(np.abs(subband)) / (R * C)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20,) + inhomogeneous part.